In [ ]:
import cv2
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import imgaug.augmenters as iaa

In [ ]:
!pip install pip install imgaug

In [ ]:
!pip install matplotlib

In [ ]:
new_data_path = Path("data/custom_camera_data")
new_data_path.mkdir(exist_ok=True)

(new_data_path / "train").mkdir(exist_ok=True)  # query and gallery are the same as the train

In [ ]:
extracted_faces = list(Path("data/camera_captures/").glob("*"))

people_num = len(extracted_faces)
people_num

In [ ]:
def augment_images(img, count):
    imgs = [img]

    for i in range(count):
        aug = iaa.Sequential([])

        rand_number = np.random.randint(0, 101)
        if rand_number < 33:
            aug.append(iaa.AdditiveGaussianNoise(loc=0, scale=(0.01*255, 0.08*255)))
        elif rand_number < 70:
            aug.append(iaa.AverageBlur(k=(3, 3)))

        rand_number = np.random.randint(0, 101)
        if rand_number < 30:
            aug.append(iaa.Multiply((0.7, 1.2)))
        elif rand_number < 70:
            aug.append(iaa.GammaContrast((1, 1.6)))            
            
        rand_number = np.random.randint(0, 101)
        if rand_number < 33:
            aug.append(iaa.ChangeColorTemperature((1100, 10000)))
        elif rand_number < 66:
            aug.append(iaa.MultiplyHueAndSaturation((0.5, 1.5), per_channel=True))           
                  
        rand_number = np.random.randint(0, 101)
        if rand_number < 33:
            aug.append(iaa.CoarseDropout(0.015, size_percent=0.1, per_channel=0.5))
        elif rand_number < 66:
            aug.append(iaa.SaltAndPepper(0.05, per_channel=True))
            
        rand_number = np.random.randint(0, 101)
        if rand_number < 50:
            aug.append(iaa.pillike.FilterEdgeEnhanceMore())

        aug.append(iaa.Fliplr(0.5))

        img_aug = aug(image=img)
        imgs.append(img_aug)

    return imgs

In [ ]:
pic_paths = list(extracted_faces[0].glob("*png"))

img = cv2.imread(pic_paths[0].as_posix())
img.shape

augmented_imgs = augment_images(img, count=7)

for img in augmented_imgs:
    plt.imshow(img[..., ::-1])
    plt.show()

In [ ]:
# add augmentations
counter = 0

for i, pid in enumerate(extracted_faces):
    pic_paths = list(pid.glob("*png"))
    for path in pic_paths:
        img = cv2.imread(path.as_posix())
        rand_number = np.random.randint(5, 11)
        
        augmented_imgs = augment_images(img, count=rand_number)
        for aug_img in augmented_imgs:
            dst = new_data_path / "train" / f"p{i}_c0_{counter}.png"
            assert(aug_img.shape[0] == 250 and aug_img.shape[1] == 250)
            cv2.imwrite(dst.as_posix(), aug_img)
            counter += 1
print("done")

In [ ]:
new_data_path = Path("data/face_data_custom")
new_data_path.mkdir(exist_ok=True)

(new_data_path / "train").mkdir(exist_ok=True)
(new_data_path / "query").mkdir(exist_ok=True)
(new_data_path / "gallery").mkdir(exist_ok=True)

In [ ]:
extracted_faces = list(Path("data/Extracted Faces/").glob("*"))

people_num = len(extracted_faces)
people_num

In [ ]:
train_perc = 0.75
train_data_len = int(people_num*train_perc)
train_data = extracted_faces[:train_data_len]

In [ ]:
counter = 0
for i, pid in enumerate(train_data):
    pic_paths = list(pid.glob("*jpg"))
    for path in pic_paths:
        src = path
        dst = new_data_path / "train" / f"p{i}_c0_{counter}.jpg"
        counter += 1
        shutil.copy(src, dst)

In [ ]:
test_data = extracted_faces[train_data_len:]

In [ ]:
for j, pid in enumerate(test_data, 1):
    pic_paths = list(pid.glob("*jpg"))
    idx = len(pic_paths) // 2

    for path in pic_paths[:idx]:
        src = path
        dst = new_data_path / "query" / f"p{i+j}_c0_{counter}.jpg"
        counter += 1
        shutil.copy(src, dst)
        
    for path in pic_paths[idx:]:
        src = path
        dst = new_data_path / "gallery" / f"p{i+j}_c0_{counter}.jpg"
        counter += 1
        shutil.copy(src, dst)

In [ ]:
import torchreid
from glob import glob
import os.path as osp

In [ ]:
class NewDataset(torchreid.data.datasets.ImageDataset):
    dataset_dir = 'data/custom_camera_data/'

    def __init__(self, **kwargs):
        self.train_dir = self.dataset_dir + "train/"  
        self.query_dir = self.dataset_dir + "train/"
        self.gallery_dir = self.dataset_dir + "train/"
        print(self.train_dir)
        input(1)
        
        train = self.process_dir(self.train_dir, isQuery=False)
        query = self.process_dir(self.query_dir, isQuery=True)
        gallery = self.process_dir(self.gallery_dir, isQuery=False)
        input(2)
        super(NewDataset, self).__init__(train, query, gallery, **kwargs)
        
    def process_dir(self, dir_path, isQuery):
        img_paths = glob(osp.join(dir_path, '*.jpg'))
        
        data = []
        for img_path in img_paths:
            img_name = img_path.split('/')[-1]
            name_splitted = img_name.split('_')
            pid = int(name_splitted[0][1:])
            camid = int(name_splitted[1][1:])

            if isQuery:
                camid += 10  # index starts from 0
            input(img_path, pid, camid)
            data.append((img_path, pid, camid))

        return data

In [ ]:
dataset = NewDataset()

In [ ]:
from pathlib import Path

In [ ]:
x = Path("data/custom_camera_data/train/")

In [ ]:
list(x.glob("*"))